# Trial 1

In [ ]:
import gc
import pickle
import warnings
from collections import defaultdict

import dpctl
import numpy as np
import pandas as pd
from joblib import parallel_backend
from sklearn import set_config
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.ensemble import (
    AdaBoostRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor,
    StackingRegressor,
)
from sklearn.feature_selection import (
    SelectFromModel,
    SelectKBest,
    SelectPercentile,
    SequentialFeatureSelector,
    mutual_info_regression,
)
from sklearn.linear_model import (
    ARDRegression,
    BayesianRidge,
    ElasticNet,
    GammaRegressor,
    Lasso,
    LassoLars,
    LassoLarsIC,
    LinearRegression,
    MultiTaskElasticNet,
    MultiTaskLasso,
    PassiveAggressiveRegressor,
    PoissonRegressor,
    QuantileRegressor,
    RANSACRegressor,
    RidgeCV,
    SGDRegressor,
    TheilSenRegressor,
    TweedieRegressor,
)
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, PowerTransformer, QuantileTransformer, StandardScaler
from sklearn.svm import SVR, LinearSVR, NuSVR
from sklearn.tree import DecisionTreeRegressor
from sklearnex import patch_sklearn
from sklearnex.ensemble import RandomForestRegressor
from tqdm import tqdm
from xgboost import XGBRFRegressor as xrb

pd.options.display.max_columns = 90
pd.options.display.max_rows = 90
warnings.filterwarnings("ignore")
set_config(display="diagram")


# def conn():
#     with open("data_np.pkl", "rb") as fp:
#         dpkl = pickle.load(fp)
#     return dpkl


# X__ = conn()


# def data_n_miss(n):
#     # data with x features simultaneously missing
#     # cdef np.ndarray t, idx
#     t = X__[:, 80]
#     idx = np.where(t == n)[0]
#     return X__[idx, :80]


# ms = data_n_miss(1)
# dcl = data_n_miss(0)
dirty_results = defaultdict(dict)
reference_metadata = defaultdict(dict)
clean_results = defaultdict(dict)


patch_sklearn()
iter_ = 10000
tol = 0.000001

# estimators = [
#     LassoLarsIC(normalize=False, precompute=True, criterion="bic"),
#     # # ARDRegression(n_iter=1000,compute_score=True,tol=tol),
#     BayesianRidge(lambda_init=0.001, n_iter=iter_, tol=tol, compute_score=True),
#     Lasso(precompute=True, max_iter=iter_, tol=tol, selection="cyclic"),
#     LassoLars(precompute=True, max_iter=iter_),
#     TweedieRegressor(power=0),
#     RANSACRegressor(
#         min_samples=500, base_estimator=LassoLarsIC(normalize=False, precompute=True, criterion="aic"), max_trials=10000
#     ),
#     ElasticNet(
#         precompute=True,
#     ),
# ]


# def feat_n_miss():
#     # Col Index of incomplete features
#     # cdef np.ndarray feed, feat_idx
#     global ms
#     feed: np.ndarray = np.isnan(ms).sum(axis=0)
#     feat_idx = np.where(feed > 0)[0]
#     return feat_idx


# def feat_x_nan_idx(feature, cc):
#     # cc data_n_miss
#     # return the indices of data missing "x" feature
#     cl_ = cc[:, feature]
#     return np.where(np.isnan(cl_))[0]


# # # clean data for reference estimation
# def get_clean_data(target_f, dirty_f_idx, size=0.5):
#     global dcl
#     clash = np.array([target_f, dirty_f_idx])
#     d_features = np.setdiff1d(np.arange(80), clash)
#     test_f = dcl[:, d_features]
#     test_t = dcl[:, target_f]
#     xt, xst, yt, yst = train_test_split(test_f, test_t, test_size=size, random_state=0, shuffle=True)
#     return xt, yt


# def gen_preprocessor():
#     preprocessor = Pipeline(
#         steps=[
#             ("scaling", StandardScaler()),
#             # (
#             #     "f_select",
#             #     SequentialFeatureSelector(
#             #         n_features_to_select=0.6,
#             #         cv=3,
#             #         estimator=ElasticNet(precompute=True),
#             #         direction="forward",
#             #         n_jobs=-1,
#             #     ),
#             # ),
#             ("f_select", Select(mutual_info_regression, percentile=50)),
#         ],
#         memory="learning/",
#     )
#     return preprocessor


# # Prepopulate Learning benchmark with complete data
# # Testing QuantileTransformer
# def clean_learning(target_f, size=0.7, exx=estimators.copy()):
#     global clean_results, dcl, reference_metadata
#     d_features = np.setdiff1d(np.arange(80), np.array(target_f))
#     xf = dcl[:, d_features]
#     yf = dcl[:, target_f]
#     X_train, X_test, y_train, y_test = train_test_split(xf, yf, test_size=size, random_state=0, shuffle=True)
#     reference_metadata[target_f] = defaultdict(dict)
#     prep = gen_preprocessor()
#     print("+++++++++++++++++++++++++++++ PREPARING DATA +++++++++++++++++++++++++++++")
#     with dpctl.device_context("gpu"):
#         with parallel_backend("loky"):
#             X_train_t = prep.fit_transform(X_train, y_train)
#     reference_metadata[target_f]["prep"] = prep
#     print("+++++++++++++++++++++++++++++ PREPARING DATA COMPLETE +++++++++++++++++++++++++++++")
#     # reference_metadata[target_f]["models"] = defaultdict(dict)
#     models_data_hub = reference_metadata[target_f]["models"]
#     models_data_hub = {}
#     with parallel_backend("multiprocessing"):
#         for est in exx:
#             est_name = est.__class__.__name__
#             # e = make_pipeline(prep, est)
#             est.fit(X_train_t, y_train)
#             X_test_t = prep.transform(X_test)
#             y_pred = est.predict(X_test_t)
#             safer = gc.collect()
#             clean_results[target_f][est_name] = mean_squared_error(y_pred, y_test)
#             safer = gc.collect()
#             models_data_hub[est_name] = est


# def dirty_df(target_f, dirty_f_idx, ms=ms):
#     # dirty_f_idx.append(target_f)
#     clash = np.array([target_f, dirty_f_idx])
#     # cdef np.ndarray clash
#     d_features = np.setdiff1d(np.arange(80), clash)
#     dirty_f_training_data_idx = feat_x_nan_idx(dirty_f_idx, ms)
#     ark = ms[dirty_f_training_data_idx, :]
#     ark = ark[:, d_features]
#     ark_target = ms[dirty_f_training_data_idx, target_f]
#     return ark, ark_target


# def roll(target_f, dirty_f_idx):
#     global estimators
#     dirty_results[target_f][dirty_f_idx] = {}
#     x_train, y_train = dirty_df(target_f, dirty_f_idx)
#     X_train, X_test, y_train, y_test = train_test_split(xf, yf, test_size=size, random_state=0, shuffle=True)
#     safer = gc.collect()
#     # workflow = make_pipeline((), estimator)
#     xtest, ytest = get_clean_data(target_f, dirty_f_idx)
#     estimators_ = estimators.copy()
#     with dpctl.device_context("gpu"):

#         for est in estimators_             with parallel_backend("multiprocessing", n_jobs=-1):
#                 workflow = make_pipeline(
#                     PowerTransformer(),
#                     SequentialFeatureSelector(
#                         n_features_to_select=0.6,
#                         cv=5,
#                         estimator=ElasticNet(precompute=True),
#                         direction="forward",
#                         n_jobs=-1,
#                     ),
#                     # SelectPercentile(mutual_info_regression,percentile=50),
#                     est,
#                     memory="learning/",
#                 )
#                 # workflow = est
#                 workflow.fit(x_train, y_train)
#                 y_pred = workflow.predict(xtest)
#                 safer = gc.collect()
#                 dirty_results[target_f][dirty_f_idx][est.__class__.__name__] = mean_squared_error(y_pred, ytest)
#                 # print(workflow.__class__.__name__,":",mean_squared_error(y_pred, ytest))


def gen_stack():
    # Category Selector
    cat_selector = make_column_selector(dtype_exclude=np.float32)
    # Number Selector
    numerical_selector = make_column_selector(dtype_exclude=np.uint8)
    # category_transformer =
    # Feature Selector
    sel = SelectFromModel(estimator=ElasticNet(precompute=True), threshold="median")
    numeric_scaler = StandardScaler()
    cat_scaler = OneHotEncoder(sparse=True)
    linear_prep = ColumnTransformer(
        transformers=[("num", numeric_scaler, numerical_selector), ("categ", cat_scaler, cat_selector)]
    )
    tree_prep = ColumnTransformer(
        transformers=[("num", numeric_scaler, numerical_selector), ("categ", OrdinalEncoder, cat_selector)]
    )
    lasso_linear_prep = ColumnTransformer(transformers=[("num", numeric_scaler, numerical_selector)])
    modis = [
        make_pipeline(lasso_linear_prep, sel, LassoLarsIC(normalize=False, precompute=True, criterion="bic")),
        make_pipeline(lasso_linear_prep, sel, ARDRegression(n_iter=1000, compute_score=True, tol=tol)),
        make_pipeline(linear_prep, sel, BayesianRidge(lambda_init=0.001, n_iter=iter_, tol=tol, compute_score=True)),
        make_pipeline(linear_prep, sel, Lasso(precompute=True, max_iter=iter_, tol=tol, selection="cyclic")),
        make_pipeline(linear_prep, sel, LassoLars(precompute=True, max_iter=iter_)),
        make_pipeline(linear_prep, sel, TweedieRegressor(power=0)),
        make_pipeline(
            linear_prep,
            sel,
            RANSACRegressor(
                min_samples=500,
                base_estimator=LassoLarsIC(normalize=False, precompute=True, criterion="aic"),
                max_trials=10000,
            ),
        ),
        make_pipeline(
            linear_prep,
            sel,
            ElasticNet(
                precompute=True,
            ),
        ),
        make_pipeline(tree_prep, sel, HistGradientBoostingRegressor(max_iter=1000, max_depth=500)),
        make_pipeline(tree_prep, sel, GradientBoostingRegressor(random_state=0, max_depth=30)),
        make_pipeline(tree_prep, sel, DecisionTreeRegressor()),
        make_pipeline(
            tree_prep,
            sel,
            ExtraTreesRegressor(n_jobs=-1),
        ),
        make_pipeline(tree_prep, sel, AdaBoostRegressor(base_estimator=Lasso(precompute=True))),
    ]
    stacked_estimators = []
    for q in modis:
        ename = q[2].__class__.__name__
        stacked_estimators.append((ename, q))
    learning_stack = StackingRegressor(estimators=stacked_estimators, cv=3, n_jobs=-1, final_estimator=RidgeCV())
    return learning_stack


with open("data.pkl", "rb") as fp:
    df = pickle.load(fp)

trd = df[df.missing_cols == 0].copy()
X_y = trd.drop(["missing_cols"], axis=1)


def save_pipeline(c, p):
    with open(f"stacking_models/stack_{c}.pkl", "wb+") as fp:
        pickle.dump(p, fp)


def get_data_feed(c):
    X = X_y.drop([c], axis=1)
    y = X_y[c]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    return X_train, X_test, y_train, y_test


# if
#
trgs = [
    "F_1_0",
    "F_1_1",
    "F_1_2",
    "F_1_3",
    "F_1_4",
    "F_1_5",
    "F_1_6",
    "F_1_7",
    "F_1_8",
    "F_1_9",
    "F_1_10",
    "F_1_11",
    "F_1_12",
    "F_1_13",
    "F_1_14",
    "F_3_0",
    "F_3_1",
    "F_3_2",
    "F_3_3",
    "F_3_4",
    "F_3_5",
    "F_3_6",
    "F_3_7",
    "F_3_8",
    "F_3_9",
    "F_3_10",
    "F_3_11",
    "F_3_12",
    "F_3_13",
    "F_3_14",
    "F_3_15",
    "F_3_16",
    "F_3_17",
    "F_3_18",
    "F_3_19",
    "F_3_20",
    "F_3_21",
    "F_3_22",
    "F_3_23",
    "F_3_24",
    "F_4_0",
    "F_4_1",
    "F_4_2",
    "F_4_3",
    "F_4_4",
    "F_4_5",
    "F_4_6",
    "F_4_7",
    "F_4_8",
    "F_4_9",
    "F_4_10",
    "F_4_11",
    "F_4_12",
    "F_4_13",
    "F_4_14",
]

start = 3
if start == 3:
    for cl in trgs:
        # with dpctl.device_context("opencl:gpu"):
        with parallel_backend("threading", n_jobs=1):
            gc.collect()
            X_train, X_test, y_train, y_test = get_data_feed(cl)
            new_stack = gen_stack()
            gc.collect()

            new_stack.fit(X_train, y_train)
            yp = new_stack.predict(X_test)
            save_pipeline(cl, new_stack)
            print(mean_squared_error(yp, y_test))
        break


	LassoLarsIC 	BayesianRidge 	Lasso 	LassoLars 	TweedieRegressor 	RANSACRegressor 	ElasticNet
0 	1.003801   	1.003801 	1.003801 	1.003801 	1.003801 	1.004282 	1.003801
1 	1.000459 	1.000494 	1.000459 	1.000459 	1.000464 	1.002849 	1.000459
2 	0.998591 	0.998694 	0.998591 	0.998591 	0.998607 	0.999959 	0.998591

In [ ]:
target_f = 0
if target_f == 9899:
    d_features = np.setdiff1d(np.arange(80), np.array(target_f))
    xf = dcl[:, d_features]
    yf = dcl[:, target_f]
    with dpctl.device_context("opencl:gpu"):
        with parallel_backend("multiprocessing"):
            X_train, X_test, y_train, y_test = train_test_split(xf, yf, test_size=0.5, random_state=0)
            # mod = xrb(
            #     n_estimators=100,
            #     learning_rate=0.001,
            #     booster="gbtree",
            #     n_jobs=-1,
            #     gamma=0.00001,
            #     random_state=0,
            #     importance_type="total_gain",
            #     num_parallel_tree=10,
            #     tree_method="approx",
            # )
            mod = HistGradientBoostingRegressor(max_leaf_nodes=900)
            work = make_pipeline(StandardScaler(), mod)
            y_pred = work.fit(X_train, y_train).predict(X_test)
            print(mean_squared_error(y_pred, y_test, squared=False))